In [1]:
# Original dictionary
original_dict = {
    'NN_layer1': 'value1',
    'NN_layer2': 'value2',
    'NN_output': 'value3'
}

# Create a new dictionary with modified keys
updated_dict = {key.replace('NN', 'DNN'): value for key, value in original_dict.items()}

print(updated_dict)


{'DNN_layer1': 'value1', 'DNN_layer2': 'value2', 'DNN_output': 'value3'}
